In [1]:
import os
os.environ["BRASA_DATA_PATH"] = "/mnt/d/brasa"

import sys
sys.path.append('..')

from datetime import datetime

import numpy as np
import pandas as pd
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pyarrow
import statsmodels.api as sm

import brasa

In [2]:
symbol = "ABEV3"
df = brasa.get_returns(["DI1T252", "DI1T126", "DAPT252", "DAPT504", "BRLUSD", symbol], start=datetime(2023, 1, 1), end=datetime(2023, 12, 31))

In [3]:
df.corr()

,ABEV3,BRLUSD,DAPT252,DAPT504,DI1T126,DI1T252
ABEV3,1.000000,0.115450,-0.059363,-0.208916,-0.006052,-0.058450
BRLUSD,0.115450,1.000000,-0.056409,-0.133168,-0.079730,0.029800
DAPT252,-0.059363,-0.056409,1.000000,0.767341,0.017529,0.137259
DAPT504,-0.208916,-0.133168,0.767341,1.000000,0.035066,0.190488
DI1T126,-0.006052,-0.079730,0.017529,0.035066,1.000000,-0.954383
DI1T252,-0.058450,0.029800,0.137259,0.190488,-0.954383,1.000000


In [4]:
sm.OLS(df[symbol], df.drop(columns=[symbol])).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  ABEV3   R-squared (uncentered):                   0.077
Model:                            OLS   Adj. R-squared (uncentered):              0.058
Method:                 Least Squares   F-statistic:                              4.061
Date:                Thu, 11 Jul 2024   Prob (F-statistic):                     0.00147
Time:                        07:23:34   Log-Likelihood:                          732.34
No. Observations:                 248   AIC:                                     -1455.
Df Residuals:                     243   BIC:                                     -1437.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
BRLUSD         0.1308      0.111      1.175      0.241      -0.088       0.350
DAPT252        3.8817      1.666      2.330      0.021       0.600       7.164
DAPT504       -5.6114      2.247     -2.497      0.013     -10.037      -1.185
DI1T126       -0.6649      0.806     -0.825      0.410      -2.253       0.923
DI1T252       -1.1318      1.292     -0.876      0.382      -3.677       1.413
==============================================================================
Omnibus:                       13.322   Durbin-Watson:                   2.186
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               29.056
Skew:                           0.179   Prob(JB):                     4.90e-07
Kurtosis:                       4.638   Cond. No.                         25.1
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:

import numpy as np
import pandas as pd
from scipy.optimize import minimize
from statsmodels.api import OLS, add_constant


def portfolio_returns(rets, weights):
    return rets @ weights


def portfolio_risk(rets, weights):
    cov = rets.cov(ddof=1)
    return np.sqrt(weights @ cov.loc[weights.index,weights.index] @ weights)


def feat_weight(feat, portfolio_returns):
    model = OLS(portfolio_returns, add_constant(feat))
    results = model.fit()
    return -results.params.iloc[1]


def new_portfolio_data(feat, feat_weight, rets, weights):
    risk_factors = rets.copy()
    risk_factors_weights = weights.copy()
    risk_factors_weights.loc[feat.name] = feat_weight
    risk_factors[feat.name] = feat
    risk = portfolio_risk(risk_factors, risk_factors_weights)
    returns = portfolio_returns(risk_factors, risk_factors_weights)
    return risk, returns, risk_factors, risk_factors_weights


def risk_variation(feats, portfolio_returns, portfolio_risk, rets, weights):
    data = []

    for feat_name in feats.columns:
        _feat_weight = feat_weight(feats[feat_name], portfolio_returns)
        _new_portfolio_risk, _new_portfolio_returns, _new_rets, _new_portfolio_weights = new_portfolio_data(feats[feat_name], _feat_weight, rets, weights)
        data.append((_new_portfolio_risk - portfolio_risk, _new_portfolio_risk, _feat_weight))

    risk_variation = pd.DataFrame(data, index=feats.columns, columns=["risk_variation", "risk", "weight"])
    return risk_variation


def residual_risk_variation(resids, weights, rets, portfolio_risk, portfolio_weights):
    risk_ports = []
    risk_variation = []

    feats_names = weights.index
    for feat_name in feats_names:
        risk, returns, aux_rets, aux_weights = new_portfolio_data(resids[feat_name], weights[feat_name], rets, portfolio_weights)
        risk_variation.append(risk - portfolio_risk)
        risk_ports.append(risk)
    
    df = pd.DataFrame({"risk_variation": risk_variation, "risk_ports": risk_ports}, index=feats_names)
    return df


def resids_ex_feat(feats, ex_feat):
    resids = {}
    betas = []
    for c in feats.columns:
        model = OLS(feats[c], add_constant(ex_feat))
        results = model.fit()
        resids[c] = results.resid

        betas.append({
            "beta_feat": results.params.iloc[1],
        })

    feats_weights = pd.DataFrame(betas, index=list(feats.columns)) * -1
    return feats_weights, pd.DataFrame(resids)


def resids_gs(feats, orth):
    resids = {}
    betas = {}
    for c in feats.columns:
        model = OLS(feats[c] - feats[c].mean(), orth - orth.mean())
        results = model.fit()
        resids[c] = results.resid

        betas[c] = results.params.to_dict()

    return betas, pd.DataFrame(resids)


def feats_weights_and_resids_ex_feat(feats, ex_feat, portfolio_returns):
    resids = {}
    betas = []
    for c in feats.columns:
        model = OLS(feats[c], add_constant(ex_feat))
        results = model.fit()
        resids[c] = results.resid

        model2 = OLS(portfolio_returns, add_constant(resids[c]))
        results2 = model2.fit()

        betas.append({
            "beta_feat": results.params.iloc[1],
            "beta_resid": results2.params.iloc[1]
        })

    feats_weights = pd.DataFrame(betas, index=list(feats.columns)) * -1
    return feats_weights, pd.DataFrame(resids)


class Portfolio:
    def __init__(self, rets, weights):
        self.rets = rets
        self.weights = weights

    @property
    def resid(self):
        return self.rets @ self.weights


def resids_gs_obj(feats, orth):
    if orth.shape[1] == 0:
        ports = {f"{c}.0":Portfolio(feats[c] - feats[c].mean(), 1) for c in feats.columns}
        return ports

    ports = {}
    for c in feats.columns:
        model = OLS(feats[c] - feats[c].mean(), orth - orth.mean())
        results = model.fit()
        rets = pd.concat([feats[c], orth], axis=1)
        weights = pd.concat(pd.Series([1], index=[c]), results.params)
        ports[f"{c}.{orth.shape[1]}"] = Portfolio(rets, weights)

    return ports


def minimize_portfolio_variance(
    boundaries: pd.DataFrame,  
    covariance_matrix: pd.DataFrame,
    x0: pd.Series = None,
    debug: bool = False,
):  
    ########## TODO: validate inputs ##########
    if not all(boundaries.columns == ["min", "max"]):
        raise ValueError("boundaries should be a dataframe with columns `min` and `max`")

    if isinstance(boundaries, pd.DataFrame):
        boundaries = boundaries.to_numpy()
    
    ########## functions ##########
    def obj_func(x):
        return x @ covariance_matrix @ x

    def jac_obj_func(x):
        return 2 * covariance_matrix @ x

    ########## initial point ##########
    if x0 is None:
        n_securities = covariance_matrix.shape[0]

        def _force_boundaries(xi):
            x = np.array([
                boundaries[i, 1] if xi[i] > boundaries[i, 1] else xi[i]
                for i in range(n_securities)
            ])
            x = np.array([
                boundaries[i, 0] if xi[i] < boundaries[i, 0] else xi[i]
                for i in range(n_securities)
            ])
            return x

        x0 = np.zeros(n_securities)
        x0 = _force_boundaries(x0)

    ########## debug ##########
    def _callback(x, *arg):
        iteration_value = np.sqrt(obj_func(x)) * np.sqrt(252)
        print("U(x): {:.2f}".format(iteration_value))
    
    __callback = _callback if debug else None

    ########## optimizer ##########
    res = minimize(
        obj_func,
        x0,
        jac=jac_obj_func,
        method="L-BFGS-B",
        callback=__callback,
        bounds=boundaries,
        options={"maxiter": 1e05, "eps": 1e-15, "ftol": 1e-15},
    )

    return pd.Series(res.x, index=covariance_matrix.index), res


class OrthogonalVariable:
    def __init__(self, *args):
        self.variable = args[0]
        self.name = self.variable.name
        if len(args) > 1:
            self.orthogonal_variables = args[1:]
        else:
            self.orthogonal_variables = None
        self._resids_gs()

    def calculate_risk_variation(self, portfolio_returns, portfolio_risk, rets, weights):
        _porfolio_weight = feat_weight(self.resid, portfolio_returns)
        _new_portfolio_risk, _new_portfolio_returns, _new_rets, _new_portfolio_weights = new_portfolio_data(self.resid, _porfolio_weight, rets, weights)
        self.portfolio_weight = _porfolio_weight
        self.risk_variation = _new_portfolio_risk - portfolio_risk
        self.new_portfolio_returns = _new_portfolio_returns
        self.new_portfolio_risk = _new_portfolio_risk
        self.new_rets = _new_rets
        self.new_portfolio_weights = _new_portfolio_weights
        self.final_weights = self.weights * self.portfolio_weight

        return self.risk_variation

    def _resids_gs(self):
        if self.orthogonal_variables is None:
            self.resid = self.variable - self.variable.mean()
            self.resid.name = self.name
            self.weights = pd.Series([1], index=[self.name])
        else:
            var = self.variable - self.variable.mean()
            orth = pd.concat([o.resid - o.resid.mean() for o in self.orthogonal_variables], axis=1)
            model = OLS(var, orth)
            results = model.fit()
            self.resid = results.resid
            self.resid.name = self.name
            names = [self.name] + [o.name for o in self.orthogonal_variables]
            weights = [1] + (- results.params).to_list()
            self.weights = pd.Series(weights, index=names)

    def _drill_down(self):
        if self.orthogonal_variables is None:
            return pd.Series({self.name: 1})
        else:
            p1 = pd.Series({self.name: 1})
            p2 = [o._drill_down() * self.weights[o.name] for o in self.orthogonal_variables]
            return pd.concat([p1, *p2], axis=1).fillna(0).sum(axis=1)
            

    def __repr__(self):
        return self.weights.__repr__()


In [6]:
feat_db = brasa.get_returns(["DI1T252", "DI1T126", "DAPT252", "DAPT504", "WDOADJ", "WINADJ"], start=datetime(2023, 1, 1), end=datetime(2023, 12, 31))

orths = []
for c in feat_db.columns:
    print(c)
    orth = OrthogonalVariable(feat_db[c], *orths)
    orths.append(orth)

pd.concat([o._drill_down() for o in orths], axis=1)

DAPT252
DAPT504
DI1T126
DI1T252
WDOADJ
WINADJ


,0,1,2,3,4,5
DAPT252,1.0,-0.730127,0.145561,0.176608,1.440369,-1.023317
DAPT504,NaN,1.000000,-0.352593,-1.096529,-2.248521,4.703289
DI1T126,NaN,NaN,1.000000,0.610634,-1.083937,0.811601
DI1T252,NaN,NaN,NaN,1.000000,-1.601278,1.460746
WDOADJ,NaN,NaN,NaN,NaN,1.000000,0.602109
WINADJ,NaN,NaN,NaN,NaN,NaN,1.000000


In [7]:
orths = [OrthogonalVariable(feat_db[c]) for c in feat_db.columns]
selected_orths = []

symbol = "SUZB3"
_rets = brasa.get_returns([symbol], start=datetime(2023, 1, 1), end=datetime(2023, 12, 31))
_portfolio_weights = pd.Series([100_000], index=[symbol])
_portfolio_returns = portfolio_returns(_rets, _portfolio_weights)
_portfolio_risk = portfolio_risk(_rets, _portfolio_weights)

display(((_portfolio_risk * np.sqrt(252)).round(2), ))

for i in range(10):
    if len(selected_orths) > 0:
        rem_feats = feat_db.loc[:, ~feat_db.columns.isin(selected_names)]
        orths = [OrthogonalVariable(rem_feats[c], *selected_orths) for c in rem_feats.columns]
    
    risk_variation = 0
    selected_orth = None
    for o in orths:
        rv = o.calculate_risk_variation(_portfolio_returns, _portfolio_risk, _rets, _portfolio_weights)
        if rv < risk_variation: #  and abs(rv) * np.sqrt(252) > 1000
            risk_variation = rv
            selected_orth = o
    if selected_orth is None:
        break
        # raise Exception("no orthogonal variable selected")
    selected_orths.append(selected_orth)
    
    _perc_risk_change = risk_variation * np.sqrt(252) / (_portfolio_risk * np.sqrt(252))
    _portfolio_returns = selected_orth.new_portfolio_returns
    _portfolio_risk = selected_orth.new_portfolio_risk
    _rets = selected_orth.new_rets
    _portfolio_weights = selected_orth.new_portfolio_weights

    # display({
    #     "feats": selected_names,
    #     "risk_variation": risk_variation.loc[selected_names[-1], "risk_variation"] * np.sqrt(252),
    #     "old_risk": _portfolio_risk * np.sqrt(252),
    #     "new_risk": risk_variation.loc[selected_names[-1], "risk"] * np.sqrt(252),
    # })

    selected_names = [o.name for o in selected_orths]
    display((" - ".join(selected_names), (100 * _perc_risk_change).round(2), (_portfolio_risk * np.sqrt(252)).round(2)))

pd.concat([o._drill_down() * o.portfolio_weight for o in selected_orths], axis=1).fillna(0).sum(axis=1)

(25199.02,)

('WINADJ', -1.66, 24781.62)

('WINADJ - WDOADJ', -1.3, 24460.26)

('WINADJ - WDOADJ - DI1T126', -0.25, 24400.0)

('WINADJ - WDOADJ - DI1T126 - DI1T252', -0.15, 24364.49)

('WINADJ - WDOADJ - DI1T126 - DI1T252 - DAPT252', -0.06, 24349.53)

('WINADJ - WDOADJ - DI1T126 - DI1T252 - DAPT252 - DAPT504', -0.01, 24347.9)

WINADJ     -42130.118648
WDOADJ     -36076.926225
DI1T126    -92506.243779
DI1T252   -115698.145974
DAPT252    106812.225274
DAPT504    -50873.509620
dtype: float64

In [8]:
pd.concat([o._drill_down() * o.portfolio_weight for o in selected_orths], axis=1).fillna(0).style.format(thousands=",", precision=2)

,0,1,2,3,4,5
WINADJ,"-26,074.90","-13,404.31",413.83,"-3,264.39",584.93,-385.28
WDOADJ,0.00,"-39,424.76","1,509.19","1,180.51",604.84,53.29
DI1T126,0.00,0.00,"-22,142.14","-60,561.62","-20,513.07","10,710.58"
DI1T252,0.00,0.00,0.00,"-99,676.82","-33,411.26","17,389.93"
DAPT252,0.00,0.00,0.00,0.00,"82,152.01","24,660.22"
DAPT504,0.00,0.00,0.00,0.00,0.00,"-50,873.51"


In [9]:
X = pd.concat([selected_orths[0].resid, selected_orths[1].resid, selected_orths[2].resid], axis=1)
# X = selected_orths[0].resid
OLS(_rets[symbol], add_constant(X)).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALE3   R-squared:                       0.312
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                     36.87
Date:                Sun, 02 Jun 2024   Prob (F-statistic):           1.10e-19
Time:                        09:13:01   Log-Likelihood:                 710.24
No. Observations:                 248   AIC:                            -1412.
Df Residuals:                     244   BIC:                            -1398.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003      0.001     -0.298      0.766      -0.002       0.001
WINADJ         0.8094      0.080     10.087      0.000       0.651       0.968
DAPT504        2.4558      1.359      1.807      0.072      -0.222       5.133
DAPT252       -4.2790      1.810     -2.364      0.019      -7.845      -0.713
==============================================================================
Omnibus:                        7.820   Durbin-Watson:                   1.789
Prob(Omnibus):                  0.020   Jarque-Bera (JB):               12.831
Skew:                           0.112   Prob(JB):                      0.00164
Kurtosis:                       4.092   Cond. No.                     2.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
X = feat_db.loc[:, ["MLCX", "SMLL", "DAPT252"]]
res = OLS(_rets["PETR4"], add_constant(X)).fit()
res.summary()

KeyError: "['MLCX', 'SMLL'] not in index"

In [ ]:
np.cov(res.resid, X.iloc[:, 2])

array([[9.77679913e-05, 5.80023693e-21],
       [5.80023693e-21, 5.51073819e-05]])